# SQL. Анализ базы данных сервиса для чтения книг по подписке

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.  

`Цель исследования`: получить данные, которые помогут сформулировать ценностное предложение для нового продукта.

## Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;  

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

![Текст с описанием картинки](https://concrete-web-bad.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F069818d1-0e5c-4d87-a461-0de584ab9c33%2FUntitled_(33).png?id=bd53c8db-b4fd-49eb-8cc9-572ebb3c9163&table=block&spaceId=9e4bd47b-c6e6-4ca3-bcee-279794b47315&width=1640&userId=&cache=v2)

## Исследуем таблицы

In [ ]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [ ]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### books

In [ ]:
query = '''

SELECT *
FROM books LIMIT 5

'''
pd.io.sql.read_sql(sql=text(query), con = engine.connect())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### authors

In [ ]:
query = '''

SELECT *
FROM authors LIMIT 5

'''
pd.io.sql.read_sql(sql=text(query), con = engine.connect())

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### publishers

In [ ]:
query = '''

SELECT *
FROM publishers LIMIT 5

'''
pd.io.sql.read_sql(sql=text(query), con = engine.connect())

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### ratings

In [ ]:
query = '''

SELECT *
FROM ratings LIMIT 5

'''
pd.io.sql.read_sql(sql=text(query), con = engine.connect())

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### reviews

In [ ]:
query = '''

SELECT *
FROM reviews LIMIT 5

'''
pd.io.sql.read_sql(sql=text(query), con = engine.connect())

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задания

### Посчитайте, сколько книг вышло после 1 января 2000 года;


In [ ]:
query = '''

SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'

'''

pd.io.sql.read_sql(sql=text(query), con = engine.connect())

,count
0,819


После `1 января 2000` года вышло `819` книг.

<div class="alert alert-block alert-success">✔️
    

__Комментарий от тимлида №1__

Расчёт верный
</div>

### Для каждой книги посчитайте количество обзоров и среднюю оценку;


In [ ]:
query = '''

SELECT b.book_id,
       b.title,
       COUNT(DISTINCT rv.review_id) AS reviews_cnt,
       ROUND(AVG(r.rating),3) AS avg_rating
FROM books AS b
LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
LEFT JOIN ratings AS r ON b.book_id = r.book_id
GROUP BY b.book_id, b.title
ORDER BY avg_rating DESC

'''
pd.io.sql.read_sql(sql=text(query), con = engine.connect())

,book_id,title,reviews_cnt,avg_rating
0,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,901,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,390,Light in August,2,5.00
3,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,316,His Excellency: George Washington,2,2.00
997,202,Drowning Ruth,3,2.00
998,371,Junky,2,2.00


Значение среднего рейтинга книг варьируется от `1.5` до `5`

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;


In [ ]:
query = '''

WITH tab AS
(SELECT publisher_id,
        book_id
FROM books
WHERE num_pages > 50)

SELECT tab.publisher_id,
       p.publisher
FROM tab
INNER JOIN publishers AS p ON tab.publisher_id = p.publisher_id
GROUP BY tab.publisher_id, p.publisher
ORDER BY COUNT(tab.book_id) DESC
LIMIT 1

'''
pd.io.sql.read_sql(sql=text(query), con = engine.connect())

,publisher_id,publisher
0,212,Penguin Books


Издательство, выпустившее наибольшее число книг толще 50 страниц - `Penguin Books`

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;


In [ ]:
query = '''

WITH tab AS
(SELECT book_id
FROM ratings
GROUP BY book_id
HAVING COUNT(rating_id) >= 50)

SELECT a.author_id,
       a.author,
       AVG(r.rating) AS avg_rating
FROM books AS b
INNER JOIN tab ON b.book_id = tab.book_id
INNER JOIN authors AS a ON b.author_id = a.author_id
INNER JOIN ratings AS r ON tab.book_id = r.book_id
GROUP BY a.author_id, a.author
ORDER BY AVG(r.rating) DESC
LIMIT 1

'''
pd.io.sql.read_sql(sql=text(query), con = engine.connect())

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097


Автор с самой высокой средней оценкой книг (среди книг с 50 и более оценками) - `J.K. Rowling/Mary GrandPré`	(коллаборация автора и иллюстратора)

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [ ]:
query = '''

SELECT COUNT(DISTINCT rv.text)/COUNT(DISTINCT rv.username) AS avg_rv_count
FROM ratings AS r
INNER JOIN reviews AS rv ON r.username = rv.username
WHERE r.username in (SELECT username
                     FROM ratings
                     GROUP BY username
                     HAVING COUNT(rating_id) > 48)

'''
pd.io.sql.read_sql(sql=text(query), con = engine.connect())

,avg_rv_count
0,24


Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок - `24 обзора`